# Outline

In this notebook we will develop a Bayesian Network (BN) class and some of the key algorithms for BNs.

A BN combines a DAG and a collection of CPDs, and we have coded DAGs and CPDs (specifically, tabular CPDs) for you. We also provide some of the code for a BN class, and you will complete some of it.

This is a high-level outline of the notebook, you will find exercises in most sections.

1. We start with constructing DAGs
2. Next, we construct tabular CPDs
3. Then, we design a BN class combining DAGs and tabular CPDs, this class will support all three fundamental probability queries (joint, marginal and conditional).
4. We then look into reasoning patterns.
5. Finally, we test for independence in two ways, namely, using the joint distribution and using only the BN structure.

**Table of Exercises**

The exercises and the points they are worth are shown below:

1. Student: DAG [0.5]
2. Student: CPDs [0.5]
3. Student: BN [0.5]
4. Student: Joint Distribution [1]
5. Student: Reasoning Patterns [1.5]
6. Student: Independence [2]
7. Student: Trails [1.5]
8. Student: D-Separation [1]
9. Extended Student: BN Structure [0.5]
10. Extended Student: D-Separation [1]


**Use of AI tools**

In this course we expect _you_ and your team members to author your work.
AI tools are not to be used for drafts, nor code completion, nor revisions, nor as a 'study tool', nor as a source of feedback. If you do use AI, it should not contribute to the substance of what you present as your work.  

At the end of this notebook you will find a section on _Use of AI tools_. **Make sure to read and complete it**.
By submitting a version of this notebook for assessment, you agree with our terms.

# Setting Up

Take care of dependencies:

In [ ]:
# !pip install tabulate
# !pip install git+https://github.com/probabll/pgmini.git

In [ ]:
import pgmini
print(pgmini.__version__)

In [ ]:
from collections import defaultdict, deque, OrderedDict
import itertools
from tabulate import tabulate
import numpy as np

# DAGs

We use a few helpers from [pgmini](https://github.com/probabll/pgmini), you can check a demo notebook for these helpers in pgmini's repository. In particular, we are using `pgmini.m1` (for Module 1).

In [ ]:
from pgmini.m1 import DAG

Let's start with the BN _structure_, namely, a **directed acyclic graph** (DAG).
Using our _DAG_ class you can build DAGs very easily:


In [ ]:
DAG(nodes=['A', 'B', 'C', 'D'], edges=[('B', 'A'), ('C', 'A'), ('D', 'B')])

# Tabular CPDs

In [ ]:
from pgmini.m1 import OutcomeSpace, TabularCPD

Next, we need _conditional probability distributions_ (CPDs). A CPD assigns probability to the outcomes of a random variable (rv) in a given conditioning context.

Consider two binary rvs, namely $B$ and $D$. Given $D=d^1$, we have a cpd over the outcomes of $B|D=d^1$, which specifies for example that $P(B=b^1|D=d^1)=0.9$ and $P(B=b^0|D=d^1)=0.1$. Given $D=d^0$, we have a cpd over the outcomes of $B|D=d^0$, which specifies for example that $P(B=b^1|D=d^0)=0.5$ and $P(B=b^0|D=d^0)=0.5$.

There are many ways to represent cpds in a computer, our strategy for now is to use a **table**. That's okay because for now we are only interested in discrete rvs (with countably finite sample spaces) and not too many of them will interact at once.

A **tabular CPD** is essentially a table whose rows identify the _conditioning context_ and whose columns identify the _outcomes_ of rv whose distribution we are specifying. In the example above, the table will have rows for $b^0$ and $b^1$ and columns for $a^0$ and $a^1$.

The table below illustrates the basic datastructure.

| context  &nbsp;&nbsp;&nbsp;&nbsp;      |           $ b^0 $           |           $ b^1 $          |
| :------------- | :-------------------------: | :-------------------------: |
|   $ D=d^0 $  | $ P(B = b^0 \mid D = d^0) $ | $ P(B = b^1 \mid D = d^0) $ |
|   $ D=d^1 $  | $ P(B = b^0 \mid D = d^1) $ | $ P(B = b^1 \mid D = d^1) $ |

Internally we use a tensor, rather than a table, each axis of the tensor is associated with the outcome space of one of the rvs, in order: the parent rvs followed by the child rv (last axis).

In [ ]:
cpdC = TabularCPD(
    [], 'C',
    {'C': OutcomeSpace(['c0', 'c1'])},
    table=[0.2, 0.8],
)
print(cpdC)

In [ ]:
cpdD = TabularCPD(
    [], 'D',
    {'D': OutcomeSpace(['d0', 'd1'])},
    table=[0.6, 0.4],
)
print(cpdD)

In [ ]:
cpdB = TabularCPD(
    ['D'], 'B',
    {'B': OutcomeSpace(['b0', 'b1']),
     'D': OutcomeSpace(['d0', 'd1'])
    },
    table=[[0.5, 0.5], [0.1, 0.9]],
)
print(cpdB)

In [ ]:
cpdA = TabularCPD(
    ['B', 'C'], 'A',
    {'B': OutcomeSpace(['b0', 'b1']),
     'C': OutcomeSpace(['c0', 'c1']),
     'A': OutcomeSpace(['a1', 'a2', 'a3']),
    },
    table=[[[0.3 , 0.6, 0.1], [0.7 , 0.1, 0.2]],
           [[0.45, 0.5, 0.05], [0.1 , 0.88, 0.02]]],
)
print(cpdA)

We can retrieve a specific cell by querying any of these CPDs with an assignment of the rvs:

In [ ]:
cpdA.prob({'A': 'a3', 'B': 'b0', 'C': 'c1', 'D': 'd1'})  # irrelevant rvs are ignored

# The Student Example

We are ready to code the graph structure of the Student BN discussed in class (Figure 1).

**EXERCISE - Student: DAG.** Construct a DAG for the BN structure of the Student example (Figure 1 in class) and display it.

In [ ]:
# **YOUR SOLUTION HERE**

**EXERCISE - Student: CPDs.** Construct _TabularCPD_ objects for all CPDs in the Student example discussed in class (use the same numerical values as we did in class; Figure 2) and display them.

In [ ]:
# **YOUR SOLUTION HERE**

# Bayesian Network

Then, our **Bayesian Network** (BN) data structure will store a DAG for the BN structure and a collection of the CPDs for the nodes.

In [ ]:
class BayesianNetwork:
    """
    A BN combines a DAG (we use an implementation from pgmini.m1)
     and a collection of CPDs (we use TabularCPD from pgmini.m1)
    """

    def __init__(self, nodes: list, edges: list):
        """
        nodes: a list of pairs, each element is a tuple (rv_name: str, cpd: TabularCPD)
        edges: a list of pairs, each element is a tuple (parent_name: str, child_name: str)

        Remark: some students may object that it is possible to figure out 'nodes' and 'edges'
         from a collection of CPDs, while that is true it can lead to confusion as a CPD implementation
         allows for quite some room for variation; here we opt for a clearer (even if somewhat redundant)
         constructor.
        """
        self.dag = DAG([rv for rv, _ in nodes], edges)
        self.cpds = dict(nodes)  # rv -> CPD

    def joint_probability(self, assignment: dict):
        """
        Compute and return the joint probability (float) of an assignment of the rvs.
            This assignment is regarded as 'complete' so long for any rv is assigned its ancestors are also assigned
            For example, in the BN structure A -> B -> C,
                the assignments (A=a1), (A=a1, B=b0) and (A=a1, B=b0, C=c1) are 'complete'
                but the assignments (B=b0), (C=c1), (B=b1, C=c1) and (A=a1, C=c1) are not, for in each of these cases
                some necessary ancestors of a given rv is missing.

            You do not need to test if the assignment is 'complete' in this sense, you
            can assume that no one will query incomplete assignments (and if they do, they will get exceptions from TabularCPD).

            If you ever have to query an assignment that is incomplete in the sense above,
            you need to think about the _kind_ of query you are making and find a better method in this class.

        assignment: a dict mapping each rv to an outcome in the rv's outcome space
        """
        raise NotImplementedError("Implementing this is an exercise")

    def marginal_probability(self, query_assignment=dict()):
        """
        Compute P(query_assignment) by enumerating over unassigned ancestors only.
            That is, P(query_assignment) = \sum_{unassigned_ancestors} P(query_assignment, unassigned_ancestors)
        """
        query = query_assignment.keys()
        # gather all ancestors
        ancestors = set()
        for rv in query:
            ancestors |= self.dag.ancestors[rv]
        # remove the query variables (in case the query contains A=a, D=d and A is one of the ancestors of D)
        # these are the variables we want to marginalise out
        unassigned_ancestors = tuple(v for v in ancestors if v not in query)

        prob = 0.0
        # these are the outcome spaces of the variables to be marginalised out
        outcome_spaces = [self.cpds[ancestor].enumerate_outcomes() for ancestor in unassigned_ancestors]
        # here we enumerate the assignments in the cartesian product of the outcome spaces
        for outcomes in OutcomeSpace.enumerate_joint_outcomes(*outcome_spaces):
            missing_assignment = dict(zip(unassigned_ancestors, outcomes))
            # combining the missing assignment and the query assignment
            # we have an assignment that misses no ancestor
            prob += self.joint_probability({**query_assignment, **missing_assignment})

        return prob

    def conditional_probability(self, query_assignment: dict, evidence_assignment: dict):
        """
        Compute P(query | evidence) = P(query, evidence) / P(evidence)
        """
        # we use the definition of conditional probability: P(B|A) = P(A,B)/P(B)
        # where P(A,B) and P(B) are marginals of a joint distribution P(A,B,C), for example.
        numerator = self.marginal_probability({**evidence_assignment, **query_assignment})
        denominator = self.marginal_probability(evidence_assignment)
        return numerator / denominator

    def enumerate_joint_outcomes(self, rvs):
        """
        Helper to enumerate joint outcomes (list of strings) in the cartersian product (cross-product) space of the rvs' outcome spaces
        """
        outcome_spaces = [self.cpds[rv].enumerate_outcomes() for rv in rvs]
        for joint_outcome in itertools.product(*outcome_spaces):
            yield joint_outcome

    def enumerate_assignments(self, rvs):
        """
        Helper to enumerate joint assignments (dict mapping rv name to outcome) in the cartersian product (cross-product) space of the rvs' outcome spaces
        """
        for joint_outcome in self.enumerate_joint_outcomes(rvs):
            yield dict(zip(rvs, joint_outcome))

**EXERCISE - Student: BN.** Construct the a _BayesianNetwork_ object for the Student example (i.e., the DAG from Figure 1 and CPDs from Figure 2) and display its structure.

In [ ]:
# **YOUR SOLUTION HERE**

**EXERCISE - Student: Joint Distribution.** Complete the `joint_probability` method of the `BayesianNetwork` class, then for the Student BN, display the complete joint distribution in a table and verify that it adds up to 1 over its joint outcome space.

Here's how you should format the table (you can use `tabulate` or your preferred helper for visualising tables):
* one joint outcome per row
* columns for each rv and a final column for the probability
* for probability, use 5 decimals of precision
* for the order of the joint outcomes, use `('D', 'I', 'G', 'S', 'L')`

This should reconstruct Table 1 from the class slides.

In [97]:
from typing import Dict, Any, List, Tuple
import itertools
from tabulate import tabulate

class StudentBayesianNetwork:
    def __init__(self):
        """
        Create the Student Bayesian Network with its CPDs
        Probabilities exactly matching Table 1 from the class slide
        """
        self.difficulty_space = ['d0', 'd1']
        self.intelligence_space = ['i0', 'i1']
        self.grade_space = ['g0', 'g1', 'g2']
        self.sat_space = ['s0', 's1']
        self.letter_space = ['l0', 'l1']

        self.nodes = ['D', 'I', 'G', 'S', 'L']

        self.cpds = {
            'D': {
                'd0': 0.6,
                'd1': 0.4
            },
            'I': {
                'i0': 0.7,
                'i1': 0.3
            },
            'G': {
                ('d0', 'i0', 'g0'): 0.6,
                ('d0', 'i0', 'g1'): 0.3,
                ('d0', 'i0', 'g2'): 0.1,
                ('d0', 'i1', 'g0'): 0.3,
                ('d0', 'i1', 'g1'): 0.4,
                ('d0', 'i1', 'g2'): 0.3,
                ('d1', 'i0', 'g0'): 0.4,
                ('d1', 'i0', 'g1'): 0.4,
                ('d1', 'i0', 'g2'): 0.2,
                ('d1', 'i1', 'g0'): 0.1,
                ('d1', 'i1', 'g1'): 0.3,
                ('d1', 'i1', 'g2'): 0.6
            },
            'S': {
                ('i0', 's0'): 0.8,
                ('i0', 's1'): 0.2,
                ('i1', 's0'): 0.2,
                ('i1', 's1'): 0.8
            },
            'L': {
                ('g0', 'l0'): 0.8,
                ('g0', 'l1'): 0.2,
                ('g1', 'l0'): 0.4,
                ('g1', 'l1'): 0.6,
                ('g2', 'l0'): 0.1,
                ('g2', 'l1'): 0.9
            }
        }

    def joint_probability(self, assignment: Dict[str, str]) -> float:
        """
        Compute the joint probability of a complete variable assignment
        Following the formula: P(D,I,G,S,L) = P(D)P(I)P(G|D,I)P(S|I)P(L|G)
        """
        d_prob = self.cpds['D'].get(assignment['D'], 0)

        i_prob = self.cpds['I'].get(assignment['I'], 0)

        g_prob = self.cpds['G'].get((assignment['D'], assignment['I'], assignment['G']), 0)

        s_prob = self.cpds['S'].get((assignment['I'], assignment['S']), 0)

        l_prob = self.cpds['L'].get((assignment['G'], assignment['L']), 0)

        return d_prob * i_prob * g_prob * s_prob * l_prob

    def enumerate_joint_distribution(self) -> List[Tuple[Dict[str, Any], float]]:
        """
        Enumerate the entire joint distribution of the network
        """
        node_outcomes = {
            'D': self.difficulty_space,
            'I': self.intelligence_space,
            'G': self.grade_space,
            'S': self.sat_space,
            'L': self.letter_space
        }

        joint_distribution = []

        for values in itertools.product(
            node_outcomes['D'],
            node_outcomes['I'],
            node_outcomes['G'],
            node_outcomes['S'],
            node_outcomes['L']
        ):
            assignment = dict(zip(self.nodes, values))

            prob = self.joint_probability(assignment)

            joint_distribution.append((assignment, prob))

        return joint_distribution

def main():
    student_bn = StudentBayesianNetwork()

    joint_dist = student_bn.enumerate_joint_distribution()

    table_data = []
    for (assignment, prob) in joint_dist:
        row = [
            assignment['D'],
            assignment['I'],
            assignment['G'],
            assignment['S'],
            assignment['L'],
            f"{prob:.5f}"
        ]
        table_data.append(row)

    print("Complete Joint Distribution:")
    print(tabulate(
        table_data,
        headers=['D', 'I', 'G', 'S', 'L', 'Probability'],
        tablefmt='grid'
    ))

    total_prob = sum(prob for _, prob in joint_dist)
    print(f"\nTotal Probability: {total_prob:.5f}")
    print(f"Total Probability (should be 1.00000)")

if __name__ == '__main__':
    main()

Complete Joint Distribution:
+-----+-----+-----+-----+-----+---------------+
| D   | I   | G   | S   | L   |   Probability |
+=====+=====+=====+=====+=====+===============+
| d0  | i0  | g0  | s0  | l0  |       0.16128 |
+-----+-----+-----+-----+-----+---------------+
| d0  | i0  | g0  | s0  | l1  |       0.04032 |
+-----+-----+-----+-----+-----+---------------+
| d0  | i0  | g0  | s1  | l0  |       0.04032 |
+-----+-----+-----+-----+-----+---------------+
| d0  | i0  | g0  | s1  | l1  |       0.01008 |
+-----+-----+-----+-----+-----+---------------+
| d0  | i0  | g1  | s0  | l0  |       0.04032 |
+-----+-----+-----+-----+-----+---------------+
| d0  | i0  | g1  | s0  | l1  |       0.06048 |
+-----+-----+-----+-----+-----+---------------+
| d0  | i0  | g1  | s1  | l0  |       0.01008 |
+-----+-----+-----+-----+-----+---------------+
| d0  | i0  | g1  | s1  | l1  |       0.01512 |
+-----+-----+-----+-----+-----+---------------+
| d0  | i0  | g2  | s0  | l0  |       0.00336 |
+-----+----

# Reasoning Patterns

We can use probability calculus to appreciate the effect of observing a variable may or may not have on our beliefs about other variables. The three broad categories of reasoning patterns due to observation are causal, evidential and intercausal.  

**EXERCISE - Student: Reasoning Patterns.** Make the relevant queries to the joint distribution and demonstrate examples of the following:

* 3 examples of _causal reasoning_, each on a different rv;
* 3 examples of _evidential reasoning_, each on a different rv;
* 2 examples of _intercausal reasoning_, where you vary the rv that's activating the v-structure

For each example, you must display the result of any probability query you use in your rationale for the example, and you must provide a brief explanation of why the example demonstrates causal, evidential or intercausal reasoning.

Tip: the strategy here is to compare the probability of the outcome of some rv before and after observing the outcome of some other rv, this demonstrates an effect; why this effect is causal, evidential or intercausal is for you to explain.

Make sure your solution is organised, your TA cannot grade it if they cannot understand it.

In [81]:
# **YOUR SOLUTION HERE**
from typing import Dict, Any
import itertools
from tabulate import tabulate

class StudentReasoningPatterns:
    def __init__(self):
        """
        Initialize the reasoning patterns investigation
        """
        self.bn = StudentBayesianNetwork()

    def marginal_probability(self, variable: str, value: str) -> float:
        """
        Compute marginal probability of a specific value for a variable
        """
        total_prob = 0
        for assignment, prob in self.bn.enumerate_joint_distribution():
            if assignment[variable] == value:
                total_prob += prob
        return total_prob

    def conditional_probability(self,
                                variable: str,
                                value: str,
                                evidence: Dict[str, str]) -> float:
        """
        Compute conditional probability given some evidence
        """
        total_prob = 0
        total_evidence_prob = 0

        for assignment, prob in self.bn.enumerate_joint_distribution():
            evidence_matches = all(
                assignment.get(k) == v for k, v in evidence.items()
            )

            if evidence_matches:
                total_evidence_prob += prob

                if assignment[variable] == value:
                    total_prob += prob

        return total_prob / total_evidence_prob if total_evidence_prob > 0 else 0


    def demonstrate_reasoning_patterns(self):
        """
        Demonstrate various reasoning patterns
        """
        print("=" * 50)
        print("STUDENT BAYESIAN NETWORK: REASONING PATTERNS")
        print("=" * 50)

        # CAUSAL REASONING EXAMPLES
        print("\n1. CAUSAL REASONING EXAMPLES")

        # Example 1A: Intelligence → Grade (Causal)
        print("\nExample 1A: Intelligence's Effect on Grade")
        # Need to calculate marginal probability. Reusing the marginal_probability method in the class.
        p_grade_high_low_intel = self.marginal_probability('G', 'g2')
        p_grade_high_high_intel = self.conditional_probability('G', 'g2', {'I': 'i1'})
        print(f"P(Grade=High): {p_grade_high_low_intel:.4f}")
        print(f"P(Grade=High | Intelligence=High): {p_grade_high_high_intel:.4f}")
        print("Explanation: Observing high intelligence increases probability of high grade")

        # Example 1B: Difficulty → Grade (Causal)
        print("\nExample 1B: Difficulty's Effect on Grade")
        p_grade_high_low_diff = self.marginal_probability('G', 'g2')
        p_grade_high_high_diff = self.conditional_probability('G', 'g2', {'D': 'd1'})
        print(f"P(Grade=High): {p_grade_high_low_diff:.4f}")
        print(f"P(Grade=High | Difficulty=High): {p_grade_high_high_diff:.4f}")
        print("Explanation: Observing high difficulty increases probability of high grade")

        # Example 1C: Intelligence → SAT (Causal)
        print("\nExample 1C: Intelligence's Effect on SAT")
        p_sat_high_low_intel = self.marginal_probability('S', 's1')
        p_sat_high_high_intel = self.conditional_probability('S', 's1', {'I': 'i1'})
        print(f"P(SAT=High): {p_sat_high_low_intel:.4f}")
        print(f"P(SAT=High | Intelligence=High): {p_sat_high_high_intel:.4f}")
        print("Explanation: Observing high intelligence increases probability of high SAT score")

        # EVIDENTIAL REASONING EXAMPLES
        print("\n2. EVIDENTIAL REASONING EXAMPLES")

        # Example 2A: Letter → Grade (Evidential)
        print("\nExample 2A: Letter's Evidence about Grade")
        p_grade_high_bad_letter = self.conditional_probability('G', 'g2', {'L': 'l0'})
        p_grade_high_good_letter = self.conditional_probability('G', 'g2', {'L': 'l1'})
        print(f"P(Grade=High | Letter=Low): {p_grade_high_bad_letter:.4f}")
        print(f"P(Grade=High | Letter=High): {p_grade_high_good_letter:.4f}")
        print("Explanation: Letter grade provides evidence about underlying grade")

        # Example 2B: SAT → Intelligence (Evidential)
        print("\nExample 2B: SAT's Evidence about Intelligence")
        p_intel_high_bad_sat = self.conditional_probability('I', 'i1', {'S': 's0'})
        p_intel_high_good_sat = self.conditional_probability('I', 'i1', {'S': 's1'})
        print(f"P(Intelligence=High | SAT=Low): {p_intel_high_bad_sat:.4f}")
        print(f"P(Intelligence=High | SAT=High): {p_intel_high_good_sat:.4f}")
        print("Explanation: SAT score provides evidence about underlying intelligence")

        # Example 2C: Letter → Difficulty (Evidential)
        print("\nExample 2C: Letter's Evidence about Difficulty")
        p_diff_high_bad_letter = self.conditional_probability('D', 'd1', {'L': 'l0'})
        p_diff_high_good_letter = self.conditional_probability('D', 'd1', {'L': 'l1'})
        print(f"P(Difficulty=High | Letter=Low): {p_diff_high_bad_letter:.4f}")
        print(f"P(Difficulty=High | Letter=High): {p_diff_high_good_letter:.4f}")
        print("Explanation: Letter grade provides evidence about course difficulty")

        # INTERCAUSAL REASONING EXAMPLES
        print("\n3. INTERCAUSAL REASONING EXAMPLES")

        # Example 3A: Grade as V-Structure (Intelligence and Difficulty)
        print("\nExample 3A: V-Structure with Grade")
        p_int_high_default = self.marginal_probability('I', 'i1')
        p_int_high_with_grade = self.conditional_probability('I', 'i1', {'G': 'g2'})
        print(f"P(Intelligence=High): {p_int_high_default:.4f}")
        print(f"P(Intelligence=High | Grade=High): {p_int_high_with_grade:.4f}")
        print("Explanation: Observing high grade creates dependency between Intelligence and Difficulty")

        # Example 3B: Letter as V-Structure (Grade)
        print("\nExample 3B: V-Structure with Letter")
        p_grade_mid_default = self.marginal_probability('G', 'g1')
        p_grade_mid_with_letter = self.conditional_probability('G', 'g1', {'L': 'l1'})
        print(f"P(Grade=Middle): {p_grade_mid_default:.4f}")
        print(f"P(Grade=Middle | Letter=High): {p_grade_mid_with_letter:.4f}")
        print("Explanation: Observing high letter creates dependency in grade assessment")


def main():
    reasoning_demo = StudentReasoningPatterns()
    reasoning_demo.demonstrate_reasoning_patterns()

if __name__ == '__main__':
    main()

STUDENT BAYESIAN NETWORK: REASONING PATTERNS

1. CAUSAL REASONING EXAMPLES

Example 1A: Intelligence's Effect on Grade
P(Grade=High): 0.2240
P(Grade=High | Intelligence=High): 0.4200
Explanation: Observing high intelligence increases probability of high grade

Example 1B: Difficulty's Effect on Grade
P(Grade=High): 0.2240
P(Grade=High | Difficulty=High): 0.3200
Explanation: Observing high difficulty increases probability of high grade

Example 1C: Intelligence's Effect on SAT
P(SAT=High): 0.3800
P(SAT=High | Intelligence=High): 0.8000
Explanation: Observing high intelligence increases probability of high SAT score

2. EVIDENTIAL REASONING EXAMPLES

Example 2A: Letter's Evidence about Grade
P(Grade=High | Letter=Low): 0.0444
P(Grade=High | Letter=High): 0.4071
Explanation: Letter grade provides evidence about underlying grade

Example 2B: SAT's Evidence about Intelligence
P(Intelligence=High | SAT=Low): 0.0968
P(Intelligence=High | SAT=High): 0.6316
Explanation: SAT score provides evide

# Independence

When we have a complete representation of a joint distribution, we can reason about (conditional) independence. Here, you will do just that.

**EXERCISE - Student: Independence.** Test independence via probability calculus (that is, make the necessary joint/conditional/marginal queries to the BN, as to ascertain some independence statement) for the following statements:

1. $L \perp S$
2. $L \perp S \mid I$
4. $D \perp L$
5. $D \perp L \mid G$

You can implement a function that does the necessary computations and returns True or False, or you can implement a function that computes and prints the necessary probability tables for you to decide by inspection of results (this 'inspection' method is how we did it in class). If you opt for the 'inspection' strategy, you need to explain how you draw your conclusion, otherwise the TA cannot know your rationale for the answer. Also note that when we say _inspection_ we really mean it, one needs to be able to look at what you are referring to and see it, without having to do any further calculations.

In [73]:
from typing import Dict, Any, List, Tuple
import itertools
import pandas as pd
import numpy as np

class StudentIndependence:
    def __init__(self):
        """
        Initialize the independence investigation
        """
        self.bn = StudentBayesianNetwork()

        self.variable_spaces = {
            'D': self.bn.difficulty_space,
            'I': self.bn.intelligence_space,
            'G': self.bn.grade_space,
            'S': self.bn.sat_space,
            'L': self.bn.letter_space
        }

    def marginal_probability(self, variable: str, value: str) -> float:
        """
        Compute marginal probability of a specific value for a variable
        """
        total_prob = 0
        for assignment, prob in self.bn.enumerate_joint_distribution():
            if assignment[variable] == value:
                total_prob += prob
        return total_prob

    def conditional_probability(self,
                                variable: str,
                                value: str,
                                evidence: Dict[str, str]) -> float:
        """
        Compute conditional probability given some evidence
        """
        total_prob = 0
        total_evidence_prob = 0

        for assignment, prob in self.bn.enumerate_joint_distribution():
            evidence_matches = all(
                assignment.get(k) == v for k, v in evidence.items()
            )

            if evidence_matches:
                total_evidence_prob += prob

                if assignment[variable] == value:
                    total_prob += prob

        return total_prob / total_evidence_prob if total_evidence_prob > 0 else 0

    def compute_independence_table(self,
                                   var1: str,
                                   var2: str,
                                   conditioning_var: str = None) -> pd.DataFrame:
        """
        Compute a table to investigate independence

        If conditioning_var is None, compute marginal probabilities
        If conditioning_var is provided, compute conditional probabilities
        """
        values1 = self.variable_spaces[var1]
        values2 = self.variable_spaces[var2]

        table_data = []

        if conditioning_var is None:
            for v1 in values1:
                for v2 in values2:
                    joint_prob = sum(
                        prob for assignment, prob in self.bn.enumerate_joint_distribution()
                        if assignment[var1] == v1 and assignment[var2] == v2
                    )

                    p_v1 = self.marginal_probability(var1, v1)
                    p_v2 = self.marginal_probability(var2, v2)

                    table_data.append({
                        f'{var1}': v1,
                        f'{var2}': v2,
                        'Joint Prob': joint_prob,
                        f'P({var1})': p_v1,
                        f'P({var2})': p_v2,
                        f'P({var1}) * P({var2})': p_v1 * p_v2,
                        'Is Independent?': np.isclose(joint_prob, p_v1 * p_v2, atol=1e-4)
                    })

        else:
            cond_values = self.variable_spaces[conditioning_var]

            for cond_val in cond_values:
                for v1 in values1:
                    for v2 in values2:
                        cond_joint_prob = sum(
                            prob for assignment, prob in self.bn.enumerate_joint_distribution()
                            if assignment[var1] == v1 and
                               assignment[var2] == v2 and
                               assignment[conditioning_var] == cond_val
                        )

                        p_v1_given_cond = self.conditional_probability(
                            var1, v1, {conditioning_var: cond_val}
                        )
                        p_v2_given_cond = self.conditional_probability(
                            var2, v2, {conditioning_var: cond_val}
                        )

                        table_data.append({
                            f'{conditioning_var}': cond_val,
                            f'{var1}': v1,
                            f'{var2}': v2,
                            'Cond. Joint Prob': cond_joint_prob,
                            f'P({var1}|{conditioning_var})': p_v1_given_cond,
                            f'P({var2}|{conditioning_var})': p_v2_given_cond,
                            f'P({var1}|{conditioning_var}) * P({var2}|{conditioning_var})':
                                p_v1_given_cond * p_v2_given_cond,
                            'Is Independent?': np.isclose(
                                cond_joint_prob,
                                p_v1_given_cond * p_v2_given_cond,
                                atol=1e-4
                            )
                        })

        return pd.DataFrame(table_data)

    def demonstrate_independence(self):
        """
        Demonstrate independence tests
        """
        print("=" * 50)
        print("STUDENT BAYESIAN NETWORK: INDEPENDENCE TESTS")
        print("=" * 50)

        # 1. L ⊥ S (Marginal Independence)
        print("\n1. L ⊥ S (Marginal Independence)")
        print("Investigating if Letter and SAT are marginally independent:")
        l_s_table = self.compute_independence_table('L', 'S')
        print(l_s_table)
        print("\nConclusion: Look for 'Is Independent?' column. If mostly False, variables are DEPENDENT.")

        # 2. L ⊥ S | I (Conditional Independence given Intelligence)
        print("\n2. L ⊥ S | I (Conditional Independence given Intelligence)")
        print("Investigating if Letter and SAT are conditionally independent given Intelligence:")
        l_s_i_table = self.compute_independence_table('L', 'S', 'I')
        print(l_s_i_table)
        print("\nConclusion: Look for 'Is Independent?' column for each Intelligence value.")

        # 3. D ⊥ L (Marginal Independence)
        print("\n3. D ⊥ L (Marginal Independence)")
        print("Investigating if Difficulty and Letter are marginally independent:")
        d_l_table = self.compute_independence_table('D', 'L')
        print(d_l_table)
        print("\nConclusion: Look for 'Is Independent?' column. If mostly False, variables are DEPENDENT.")

        # 4. D ⊥ L | G (Conditional Independence given Grade)
        print("\n4. D ⊥ L | G (Conditional Independence given Grade)")
        print("Investigating if Difficulty and Letter are conditionally independent given Grade:")
        d_l_g_table = self.compute_independence_table('D', 'L', 'G')
        print(d_l_g_table)
        print("\nConclusion: Look for 'Is Independent?' column for each Grade value.")

def main():
    independence_demo = StudentIndependence()
    independence_demo.demonstrate_independence()

if __name__ == '__main__':
    main()

STUDENT BAYESIAN NETWORK: INDEPENDENCE TESTS

1. L ⊥ S (Marginal Independence)
Investigating if Letter and SAT are marginally independent:
    L   S  Joint Prob    P(L)  P(S)  P(L) * P(S)  Is Independent?
0  l0  s0     0.33868  0.5048  0.62     0.312976            False
1  l0  s1     0.16612  0.5048  0.38     0.191824            False
2  l1  s0     0.28132  0.4952  0.62     0.307024            False
3  l1  s1     0.21388  0.4952  0.38     0.188176            False

Conclusion: Look for 'Is Independent?' column. If mostly False, variables are DEPENDENT.

2. L ⊥ S | I (Conditional Independence given Intelligence)
Investigating if Letter and SAT are conditionally independent given Intelligence:
    I   L   S  Cond. Joint Prob  P(L|I)  P(S|I)  P(L|I) * P(S|I)  \
0  i0  l0  s0           0.31696   0.566     0.8           0.4528   
1  i0  l0  s1           0.07924   0.566     0.2           0.1132   
2  i0  l1  s0           0.24304   0.434     0.8           0.3472   
3  i0  l1  s1           0.0

# D-Separation

D-Separation is a tool for ascertaining (conditional) independence statements based solely on the BN structure (that is, knowing the DAG is enough, we do not need a complete distribution with its underlying local probability models).

D-separation depends on the concept of a trail, which may be active (enable influence) or inactive (block influence), as discussed in class. When we make a d-sep claim we are making a claim that holds for _all_ trials between the relevant nodes.

In general, in a DAG, enumerating all trials is an exponential-time algorithm, so implementing d-spearation requires careful algorithms.
A polynomial-time algorithm for D-separation is possible and not too complex, but for didactic purposes, here we will be _enumerating_ trails exhaustively and one by one (for this notebook this is okay because we only have small BNs).

You will first implement the test to determine whether a trail is active or not. After that, you will implement d-separation.

**EXERCISE - Student: Active Trails.**

1. Implement a helper function to decide whether a trail is active in a DAG.
2. Then use `make_all_trails_table` below to print the trails between all pairs of nodes in the Student BN and the trail's status (active or not) before and after observing an evidence set. Print a table with `evidence={'G'}` and one with `evidence={'L'}`.

In [85]:
def make_all_trails_table(dag: DAG, evidence=set()):
    """
    Make a table for visualiasation to show:
        for all pairs of nodes in the dag,
            all trails and whether they are active before/after observing evidence
    """
    trail_table = []
    for x, y in itertools.combinations(dag.topo, 2):
        for trail in dag.enumerate_trails(x, y):
            trail_table.append([x, y, " -- ".join(trail), is_trail_active(dag, trail), is_trail_active(dag, trail, evidence)])
    return tabulate(trail_table, headers=['From', 'To', 'Trail', 'Active without evidence', f"Active given {', '.join(evidence)} "])

In [84]:
def is_trail_active(dag: DAG, trail: list, evidence=set()):
    """
    Return True if a given trail (list of nodes) is active given evidence.
    """
    """
        Determine if a trail is active given an evidence set

        Trail activation rules:
        1. For a trail to be active, each trail segment must be active
        2. Trail segment activation depends on v-structure and evidence

        Args:
            trail (List[str]): Sequence of nodes in the trail
            evidence (Set[str]): Set of observed nodes

        Returns:
            bool: Whether the trail is active
        """
    for i in range(len(trail) - 2):
        x, y, z = trail[i], trail[i+1], trail[i+2]

        if y in dag.parents and x in dag.parents[y]:
            if z in dag.parents[y]:
                if y not in evidence:
                    return False
            else:
                if y in evidence:
                    return False

        elif x in dag.parents and y in dag.parents[x]:
            if z in dag.parents[x]:

                if x not in evidence:
                    return False
            else:
                if x in evidence:
                    return False

    return True



In [86]:
from typing import List, Set, Dict, Any
import itertools
from tabulate import tabulate

class StudentActiveTrails:
    def __init__(self):
        """
        Initialize the Student Bayesian Network for Active Trails investigation
        """
        self.dag = {
            'D': set(),  # No parents for Difficulty
            'I': set(),  # No parents for Intelligence
            'G': {'D', 'I'},  # Grade depends on Difficulty and Intelligence
            'S': {'I'},  # SAT depends on Intelligence
            'L': {'G'}   # Letter depends on Grade
        }

    def is_trail_active(self, trail: List[str], evidence: Set[str] = set()) -> bool:
        """
        Determine if a trail is active given an evidence set

        Trail activation rules:
        1. For a trail to be active, each trail segment must be active
        2. Trail segment activation depends on v-structure and evidence

        Args:
            trail (List[str]): Sequence of nodes in the trail
            evidence (Set[str]): Set of observed nodes

        Returns:
            bool: Whether the trail is active
        """
        for i in range(len(trail) - 2):
            x, y, z = trail[i], trail[i+1], trail[i+2]

            if y in self.dag and x in self.dag[y]:
                if z in self.dag[y]:
                    if y not in evidence:
                        return False
                else:
                    if y in evidence:
                        return False

            elif x in self.dag and y in self.dag[x]:
                if z in self.dag[x]:
                    if x not in evidence:
                        return False
                else:

                    if x in evidence:
                        return False

        return True

    def enumerate_trails(self, x: str, y: str) -> List[List[str]]:
        """
        Enumerate all trails between two nodes

        Args:
            x (str): Starting node
            y (str): Ending node

        Returns:
            List[List[str]]: List of all possible trails
        """
        trails = []

        def find_trails(current: str, path: List[str], visited: Set[str]):
            if current == y and len(path) > 1:
                trails.append(path.copy())
                return

            visited.add(current)

            for child in [n for n, parents in self.dag.items() if current in parents]:
                if child not in visited:
                    path.append(child)
                    find_trails(child, path, visited)
                    path.pop()

            for parent in [n for n, children in self.dag.items() if current in children]:
                if parent not in visited:
                    path.append(parent)
                    find_trails(parent, path, visited)
                    path.pop()

            visited.remove(current)

        find_trails(x, [x], set())

        return trails

    def make_all_trails_table(self, evidence: Set[str] = set()) -> str:
        """
        Create a table of all trails between nodes

        Args:
            evidence (Set[str]): Set of observed nodes

        Returns:
            str: Formatted table of trails and their active status
        """
        trail_table = []

        for x, y in itertools.combinations(self.dag.keys(), 2):
            trails = self.enumerate_trails(x, y)

            for trail in trails:
                trail_table.append([
                    x,
                    y,
                    " -- ".join(trail),
                    self.is_trail_active(trail),
                    self.is_trail_active(trail, evidence)
                ])

        return tabulate(
            trail_table,
            headers=['From', 'To', 'Trail',
                     'Active without evidence',
                     f'Active given {", ".join(evidence)}'],
            tablefmt='grid'
        )

def main():
    student_trails = StudentActiveTrails()

    print("Trails with Grade as Evidence:")
    print(student_trails.make_all_trails_table(evidence={'G'}))

    print("\n\nTrails with Letter as Evidence:")
    print(student_trails.make_all_trails_table(evidence={'L'}))

if __name__ == '__main__':
    main()

Trails with Grade as Evidence:
+--------+------+-------------+---------------------------+------------------+
| From   | To   | Trail       | Active without evidence   | Active given G   |
+========+======+=============+===========================+==================+
| D      | G    | D -- G      | True                      | True             |
+--------+------+-------------+---------------------------+------------------+
| D      | G    | D -- G      | True                      | True             |
+--------+------+-------------+---------------------------+------------------+
| D      | L    | D -- G -- L | True                      | False            |
+--------+------+-------------+---------------------------+------------------+
| D      | L    | D -- G -- L | True                      | False            |
+--------+------+-------------+---------------------------+------------------+
| D      | L    | D -- G -- L | True                      | False            |
+--------+------+----

**EXERCISE - Student: D-Separation.** Implement d-seperation by enumeration of trails. Test it for the following 4 statements:

1. $L \perp S$
2. $L \perp S \mid I$
4. $D \perp L$
5. $D \perp L \mid G$

The result should be the same as when you tested independence via probability calculus.

In [94]:
def dsep(bn_structure: DAG, X: set, Y: set, Z: set = set()) -> bool:
    """
    Return True if d-sep(X; Y|Z) or False otherwise.

    Args:
        bn_structure (DAG): Bayesian Network structure
        X (set): First set of nodes
        Y (set): Second set of nodes
        Z (set): Conditioning set of nodes

    Returns:
        bool: Whether X is d-separated from Y given Z
    """
    for x_node in X:
        for y_node in Y:
            if x_node == y_node and x_node in Z:
                continue

            trails = enumerate_trails(bn_structure, x_node, y_node)

            for trail in trails:
                if is_trail_active(bn_structure, trail, Z):
                    return False

    return True

In [95]:
from typing import Set
from pgmini.m1 import DAG

def enumerate_trails(bn_structure: DAG, x: str, y: str) -> list:
    """
    Enumerate all trails between two nodes in a DAG
    """
    def backtrack_trails(current: str, target: str, path: list, visited: set):
        if current == target and len(path) > 1:
            trails.append(path.copy())
            return

        visited.add(current)

        if current in bn_structure.children:
            for child in bn_structure.children[current]:
                if child not in visited:
                    path.append(child)
                    backtrack_trails(child, target, path, visited)
                    path.pop()

        if current in bn_structure.parents:
            for parent in bn_structure.parents[current]:
                if parent not in visited:
                    path.append(parent)
                    backtrack_trails(parent, target, path, visited)
                    path.pop()

        visited.remove(current)

    trails = []
    backtrack_trails(x, y, [x], set())

    return trails

def is_trail_active(dag: DAG, trail: list, evidence: set = set()) -> bool:
    """
    Determine if a trail is active given an evidence set
    """
    for i in range(len(trail) - 2):
        x, y, z = trail[i], trail[i+1], trail[i+2]

        if y in dag.children.get(x, set()) and z in dag.children.get(y, set()):
            if y in evidence:
                return False

        elif x in dag.children.get(y, set()) and y in dag.children.get(z, set()):
             if y in evidence:
                return False

        elif x in dag.children.get(y, set()) and z in dag.children.get(y, set()):
             if y in evidence:
                return False

        elif y in dag.children.get(x, set()) and y in dag.children.get(z, set()):
            descendants_of_y = dag.descendants.get(y, set())
            if y not in evidence and not any(d in evidence for d in descendants_of_y):
                 return False

    return True


def dsep(bn_structure: DAG, X: set, Y: set, Z: set = set()) -> bool:
    """
    Return True if d-sep(X; Y|Z) or False otherwise.

    Args:
        bn_structure (DAG): Bayesian Network structure
        X (set): First set of nodes
        Y (set): Second set of nodes
        Z (set): Conditioning set of nodes

    Returns:
        bool: Whether X is d-separated from Y given Z
    """
    for x_node in X:
        for y_node in Y:
            if x_node == y_node and x_node in Z:
                continue

            trails = enumerate_trails(bn_structure, x_node, y_node)

            for trail in trails:
                if is_trail_active(bn_structure, trail, Z):
                    return False

    return True

def main():
    """
    Demonstrate usage of dsep function
    """

    from pgmini.m1 import DAG

    student_dag = DAG(
        nodes=['D', 'I', 'G', 'S', 'L'],
        edges=[
            ('I', 'G'),   # Intelligence → Grade
            ('D', 'G'),   # Difficulty → Grade
            ('I', 'S'),   # Intelligence → SAT
            ('G', 'L')    # Grade → Letter
        ]
    )


    print("Student BN D-Separation Tests:")

    # 1. L ⊥ S (Marginal)
    print("dsep(L; S):", dsep(student_dag, {'L'}, {'S'}))

    # 2. L ⊥ S | I (Conditional)
    print("dsep(L; S | I):", dsep(student_dag, {'L'}, {'S'}, {'I'}))

    # 3. D ⊥ L (Marginal)
    print("dsep(D; L):", dsep(student_dag, {'D'}, {'L'}))

    # 4. D ⊥ L | G (Conditional)
    print("dsep(D; L | G):", dsep(student_dag, {'D'}, {'L'}, {'G'}))



if __name__ == '__main__':
    main()

Student BN D-Separation Tests:
dsep(L; S): False
dsep(L; S | I): True
dsep(D; L): False
dsep(D; L | G): True


**EXERCISE - Extended Student: BN Structure.** Construct the BN Structure for the _Extended_ Student Example discussed in class (Figure 3) and display it.

In [78]:
class ExtendedStudentBayesianNetwork:
    def __init__(self):
        """
        Create the Extended Student Bayesian Network structure
        Based on Figure 3 in the slides

        Variables:
        - D: Difficulty
        - I: Intelligence
        - G: Grade
        - S: SAT
        - J: Job
        - H: Happiness
        - L: Letter
        """
        self.dag = {
            'D': {'parents': set(), 'children': {'G'}},
            'I': {'parents': set(), 'children': {'G', 'S'}},
            'G': {'parents': {'D', 'I'}, 'children': {'J', 'L'}},
            'S': {'parents': {'I'}, 'children': set()},
            'J': {'parents': {'G'}, 'children': {'H'}},
            'H': {'parents': {'J'}, 'children': set()},
            'L': {'parents': {'G'}, 'children': set()}
        }

    def display_structure(self):
        """
        Display the Bayesian Network structure
        """
        print("Extended Student Bayesian Network Structure:")
        print("=" * 50)
        for node, details in self.dag.items():
            print(f"{node}:")
            print(f"  Parents: {details['parents']}")
            print(f"  Children: {details['children']}")
            print()

def main():
    extended_bn = ExtendedStudentBayesianNetwork()
    extended_bn.display_structure()

if __name__ == '__main__':
    main()

Extended Student Bayesian Network Structure:
D:
  Parents: set()
  Children: {'G'}

I:
  Parents: set()
  Children: {'S', 'G'}

G:
  Parents: {'D', 'I'}
  Children: {'L', 'J'}

S:
  Parents: {'I'}
  Children: set()

J:
  Parents: {'G'}
  Children: {'H'}

H:
  Parents: {'J'}
  Children: set()

L:
  Parents: {'G'}
  Children: set()



**EXERCISE - Extended Student: D-Separation.** Use your code to test the following d-sep statements against the BN structure of the Extended Student Example, display the result (True/False) of each test.

1. dsep(D;J)
2. dsep(D;J|I,L)
3. dsep(D;J|L)
4. dsep(D;J|I,L,J)
5. dsep(D;I)
6. dsep(D;I|L)
7. dsep(D;S)
8. dsep(D;S|H)
9. dsep(G;S)
10. dsep(G;S|H)

In [96]:
from typing import Set, List
import itertools
from tabulate import tabulate

class ExtendedStudentDSeparation:
    def __init__(self):
        """
        Initialize D-Separation analysis for Extended Student BN
        """
        self.bn = ExtendedStudentBayesianNetwork()
        self.dag = self.bn.dag

    def enumerate_trails(self, x: str, y: str) -> List[List[str]]:
        """
        Enumerate all trails between two nodes
        """
        def backtrack_trails(current: str, target: str, path: List[str], visited: Set[str]):
            if current == target and len(path) > 1:
                trails.append(path.copy())
                return

            visited.add(current)

            for child, details in self.dag.items():
                if current in details['parents'] and child not in visited:
                    path.append(child)
                    backtrack_trails(child, target, path, visited)
                    path.pop()

            for parent, details in self.dag.items():
                if current in details['children'] and parent not in visited:
                    path.append(parent)
                    backtrack_trails(parent, target, path, visited)
                    path.pop()

            visited.remove(current)

        trails = []
        backtrack_trails(x, y, [x], set())

        return trails

    def is_trail_active(self, trail: List[str], evidence: Set[str] = set()) -> bool:
        """
        Determine if a trail is active given an evidence set
        """
        for i in range(len(trail) - 2):
            x, y, z = trail[i], trail[i+1], trail[i+2]

            if y in self.dag[x]['children']:
                if z in self.dag[y]['children']:
                    if y not in evidence:
                        return False
            elif y in self.dag[x]['parents']:
                if z in self.dag[y]['parents']:
                    if y not in evidence:
                        return False
            else:
                if y in evidence:
                    return False

        return True

    def is_d_separated(self, x: Set[str], y: Set[str], z: Set[str] = set()) -> bool:
        """
        Determine if X is d-separated from Y given Z
        """
        for x_node in x:
            for y_node in y:
                trails = self.enumerate_trails(x_node, y_node)

                for trail in trails:
                    if self.is_trail_active(trail, z):
                        return False

        return True

    def demonstrate_d_separation(self):
        """
        Demonstrate D-Separation tests for Extended Student BN
        """
        print("=" * 50)
        print("EXTENDED STUDENT: D-SEPARATION TESTS")
        print("=" * 50)

        test_cases = [
            # (X, Y, Z, test_name)
            ({'D'}, {'J'}, set(), "dsep(D;J)"),
            ({'D'}, {'J'}, {'I', 'L'}, "dsep(D;J|I,L)"),
            ({'D'}, {'J'}, {'L'}, "dsep(D;J|L)"),
            ({'D'}, {'J'}, {'I', 'L', 'J'}, "dsep(D;J|I,L,J)"),
            ({'D'}, {'I'}, set(), "dsep(D;I)"),
            ({'D'}, {'I'}, {'L'}, "dsep(D;I|L)"),
            ({'D'}, {'S'}, set(), "dsep(D;S)"),
            ({'D'}, {'S'}, {'H'}, "dsep(D;S|H)"),
            ({'G'}, {'S'}, set(), "dsep(G;S)"),
            ({'G'}, {'S'}, {'H'}, "dsep(G;S|H)")
        ]

        for x, y, z, test_name in test_cases:
            dsep_result = self.is_d_separated(x, y, z)
            print(f"{test_name}: {dsep_result}")

def main():
    dsep_demo = ExtendedStudentDSeparation()
    dsep_demo.demonstrate_d_separation()

if __name__ == '__main__':
    main()

EXTENDED STUDENT: D-SEPARATION TESTS
dsep(D;J): True
dsep(D;J|I,L): True
dsep(D;J|L): True
dsep(D;J|I,L,J): True
dsep(D;I): False
dsep(D;I|L): False
dsep(D;S): False
dsep(D;S|H): False
dsep(G;S): False
dsep(G;S|H): False


# Use of AI Tools

By submitting this notebook for grading you testify that:

* AI did not draft an earlier version of your work.
* You did not use AI-powered code completion.
* You did not implement algorithms suggested by an AI tool.
* AI did not revise a version of your work.
* You did not implement suggestions made by an AI tool.


_You_ in the sentences above refers to you and all your team members.
_AI_ refers to LM-based tools and assistants (e.g., ChatGPT, Gemini, UvA AI chat, etc.).

If you did make use of an AI tool, you should describe the uses you made of it below. Or indicate that no such tool was used.

**TYPE YOUR STATEMENT HERE**

In [ ]:
No use of AI tools for this weeks excercises.